In [1]:
import os
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import backend
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras import regularizers


In [2]:
from encoders import EncoderResNet18, EncoderResNet34, EncoderResNet50, encoderCNN, EncoderMixNet18
from decoders import DecoderResNet18, DecoderResNet34, DecoderResNet50, decoderCNN
from datasets import data_loader
from embeddings import embedding
from reconstructions import reconstructions
from generations import Generations
from activations import VisualizeActivations
from gradcam import GradCam
from src.CVAE import CVAE

# import importlib
# importlib.reload(embeddings)
# from embeddings import embedding

backend.clear_session()

In [3]:
# TO DO: this should be passed as arguments
dataset_name = 'celeba'
model_name = 'CVAE'
encoded_dim = 4096
learning_rate = .00005
epoch_count = 70
batch_size = 100
patience = 5


In [4]:
if dataset_name == 'experimental':
    #TO DO: move datasets in the repo and change root_folder

    train_ds, val_ds, input_shape, category_count, labels = data_loader(name=dataset_name, root_folder='/home/PERSONALE/nicolas.derus2/HistoDL/datasets/')
else:
    #TO DO: move datasets in the repo and change root_folder

    train_x, test_x, val_x, train_y, test_y, val_y, train_y_one_hot, test_y_one_hot, val_y_one_hot, input_shape, category_count, labels = data_loader(name=dataset_name,
                                                                                                                                        root_folder='/home/PERSONALE/nicolas.derus2/HistoDL/datasets/')

In [5]:
kl_coefficient = (2*input_shape[1]/encoded_dim)**2
kl_coefficient

784.0

In [6]:
%env "WANDB_NOTEBOOK_NAME" "train.ipynb"

env: "WANDB_NOTEBOOK_NAME"="train.ipynb"


In [7]:

wandb.init(project=dataset_name, entity="nrderus",
  config = {
  "dataset": dataset_name,
  "model": model_name,
  "encoded_dim": encoded_dim,
  "kl_coefficient": kl_coefficient,
  "learning_rate": learning_rate,
  "epochs": epoch_count,
  "batch_size": batch_size,
  "patience": patience,
})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nrderus. Use `wandb login --relogin` to force relogin


In [8]:
if 'resnet' in model_name:
    encoder = EncoderResNet18(encoded_dim = encoded_dim)
    encoder = encoder.model(input_shape=(input_shape[0], input_shape[1], input_shape[2] + category_count))

else:
    #encoder = encoderCNN(input_shape, category_count, encoded_dim,  regularizer=regularizers.L2(.001))
    encoder = EncoderMixNet18(encoded_dim = encoded_dim)
    encoder = encoder.model(input_shape=(input_shape[0], input_shape[1], input_shape[2] + category_count))

encoder.summary()

2022-07-29 23:24:08.290959: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 28, 28, 11)  0           []                               
                                ]                                                                 
                                                                                                  
 layer0 (Sequential)            (None, 28, 28, 16)   1664        ['input[0][0]']                  
                                                                                                  
 layer1 (Sequential)            (None, 28, 28, 16)   2384        ['layer0[0][0]']                 
                                                                                                  
 layer2 (Sequential)            (None, 28, 28, 32)   4768        ['layer1[0][0]']           

In [9]:

encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 28, 28, 11)  0           []                               
                                ]                                                                 
                                                                                                  
 layer0 (Sequential)            (None, 28, 28, 16)   1664        ['input[0][0]']                  
                                                                                                  
 layer1 (Sequential)            (None, 28, 28, 16)   2384        ['layer0[0][0]']                 
                                                                                                  
 layer2 (Sequential)            (None, 28, 28, 32)   4768        ['layer1[0][0]']           

In [10]:
if 'resnet' in model_name:
    decoder = DecoderResNet18( encoded_dim = encoded_dim, final_stride = 2)
    decoder = decoder.model(input_shape=(encoded_dim + category_count,))
else:
    decoder = decoderCNN(input_shape, category_count, encoded_dim, final_stride = 1, regularizer=regularizers.L2(.001))

decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 12)]              0         
                                                                 
 dense_2 (Dense)             (None, 12544)             163072    
                                                                 
 reshape (Reshape)           (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_5 (Batc  (None, 14, 14, 64)       256       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 14, 14, 64)        0         
                                                                 
 up_block4_conv1 (Conv2DTran  (None, 14, 14, 64)       36928     
 spose)                                                    

In [11]:
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
    # This is the TPU initialization code that has to be at the beginning.
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))

    strategy = tf.distribute.experimental.TPUStrategy(resolver)
    with strategy.scope():
        cvae = CVAE(encoder, decoder, kl_coefficient, input_shape, category_count)
        cvae.built = True
        cvae_input = cvae.encoder.input[0]
        cvae_output = cvae.decoder.output
        mu = cvae.encoder.get_layer('mu').output
        log_var = cvae.encoder.get_layer('log_var').output

        def scheduler(epoch, lr, lr_fac=0.5, lr_epochs=60 ):
            return lr * tf.math.pow(lr_fac, tf.math.floor(float(epoch) / float(lr_epochs)))
            
        opt = keras.optimizers.Adam(learning_rate = learning_rate)
        cvae.compile(optimizer = opt, run_eagerly=False)
except:
    from src.CVAE import CVAE_balancing
    cvae = CVAE_balancing(encoder, decoder, kl_coefficient, input_shape, category_count)
    cvae.built = True
    cvae_input = cvae.encoder.input[0]
    cvae_output = cvae.decoder.output
    mu = cvae.encoder.get_layer('mu').output
    log_var = cvae.encoder.get_layer('log_var').output

    def scheduler(epoch, lr, lr_fac=0.5, lr_epochs=60 ):
        return lr * tf.math.pow(lr_fac, tf.math.floor(float(epoch) / float(lr_epochs)))
    opt = keras.optimizers.Adam(learning_rate = learning_rate)
    cvae.compile(optimizer = opt, run_eagerly=False)

In [12]:
# early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',
#              patience=patience, restore_best_weights=False)


lr_decay = tf.keras.callbacks.LearningRateScheduler(scheduler)

history = cvae.fit([train_x, train_y_one_hot],
                   validation_data = ([val_x, val_y_one_hot],None),
                   epochs = epoch_count,
                   batch_size = batch_size,
                   callbacks=[lr_decay, WandbCallback(save_model = False) ]) #save_weights_only -> ValueError: Unable to create dataset (name already exists)

wandb: WARNING Unable to compute FLOPs for this model.


Epoch 1/2
500/500 [==============================] - 75s 146ms/step - loss: 2.9716 - reconstruction_loss: 2.8667 - kl_loss: 0.1049 - loss_no_weights: 179.6421 - val_loss: 179.5422 - val_reconstruction_loss: 179.5237 - val_kl_loss: 0.0185 - val_loss_no_weights: 179.5237 - lr: 1.0000e-04 - _timestamp: 1659129924.0000 - _runtime: 81.0000
Epoch 2/2
500/500 [==============================] - 69s 138ms/step - loss: 2.6283 - reconstruction_loss: 2.6157 - kl_loss: 0.0126 - loss_no_weights: 166.6902 - val_loss: 166.0855 - val_reconstruction_loss: 166.0773 - val_kl_loss: 0.0083 - val_loss_no_weights: 166.0773 - lr: 1.0000e-04 - _timestamp: 1659129993.0000 - _runtime: 150.0000


In [13]:
tf.print(cvae.gamma_x)


1.48156321


In [14]:
tf.math.pow(lr_fac, tf.math.floor(float(epoch) / float(lr_epochs)))

NameError: name 'lr_fac' is not defined

In [ ]:
epoch=100

tf.math.pow(0.5, tf.math.floor(float(epoch) / float(60)))

In [ ]:
#encoder2
import keras.layers as layers
z_cond = layers.Input(shape=(100 + label_size,), dtype='float32',
            name='Input')
t = layers.Dense(self.second_dim, name='fc'+'0')(z_cond)
t = layers.LeakyReLU(0.2)(t)
for i in range(self.second_depth - 1):
    t = layers.Dense( self.second_dim, tf.nn.relu, name='fc'+str(i))(t)
    t = layers.LeakyReLU(0.2)(t)
t = layers.Concatenate(axis=-1)([z_cond, t]) 

self.u_mean = layers.Dense( self.latent_dim, name='mu_u')(t)
self.u_log_var = layers.Dense( self.latent_dim, name='logsd_u')(t)
self.u_var = tf.exp(self.u_log_var)
#self.u = self.mu_u + self.sd_u * tf.random_normal([self.batch_size, self.latent_dim]) #reparametrizaiton
self.u = self.sampling(u_mean, u_log_var, input_label)

In [ ]:
#decoder2

u_cond = layers.Input(shape=(100 + label_size,), dtype='float32',
            name='Input')

u = layers.Dense(self.second_dim, name='fc'+'0')(u_cond)
u = layers.LeakyReLU(0.2)(u)

for i in range(self.second_depth):
    u = layers.Dense( self.second_dim, tf.nn.relu, name='fc'+str(i))(u)
    u = layers.LeakyReLU(0.2)(u)

u = layers.Concatenate(axis=-1)([u_cond, u]) 
self.z_hat = layers.Dense(self.latent_dim, name='z_hat')(u)


In [ ]:
 @tf.function
    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            input_img, input_label, conditional_input = self.conditional_input(data)
            z_mean, z_log_var = self.encoder(conditional_input)
            z_cond = self.sampling(z_mean, z_log_var, input_label)
            reconstruction = self.decoder(z_cond)

            reconstruction_loss = tf.reduce_sum(
                 keras.losses.MSE(input_img, 
                                    reconstruction), axis=(1, 2))            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean)
                      - tf.exp(z_log_var))

            kl_loss = tf.reduce_sum(kl_loss, axis=1) 
            total_loss_no_weights = reconstruction_loss + kl_loss
            total_loss_no_weights = tf.reduce_mean(total_loss_no_weights)
            kl_loss = self.beta * kl_loss
            HALF_LOG_TWO_PI = tf.constant(0.91893,  dtype=tf.float32)# added
            reconstruction_loss = reconstruction_loss / self.gamma_x + self.loggamma_x + HALF_LOG_TWO_PI #added
            if self.reconstruction_loss_tracker.result() > 0:
                reconstruction_loss = tf.minimum(self.reconstruction_loss_tracker.result(), self.reconstruction_loss_tracker.result()*.99 + reconstruction_loss *.01) #min between cumulated reconstruction loss and this batch.
            total_loss = reconstruction_loss + kl_loss
            total_loss = tf.reduce_mean(total_loss)

        self.gamma_x.assign( tf.sqrt(tf.reduce_mean(reconstruction_loss)))#added
        self.loggamma_x.assign( tf.math.log(self.gamma_x)) #ådded

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.total_loss_no_weights_tracker.update_state(total_loss_no_weights)
        #tf.print(self.gamma_x)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
            "loss_no_weights": self.total_loss_no_weights_tracker.result(),

        }


In [ ]:
# tf.saved_model.save(cvae.encoder, 'cvae_encoder')
# tf.saved_model.save(cvae.decoder, 'cvae_decoder')

In [ ]:
_, input_label_train, train_input = cvae.conditional_input([train_x[:1000], train_y_one_hot[:1000]])
_, input_label_test, test_input = cvae.conditional_input([test_x[:1000], test_y_one_hot[:1000]])
_, input_label_val, val_input = cvae.conditional_input([val_x[:1000], val_y_one_hot[:1000]])

train_x_mean, train_log_var = cvae.encoder.predict(train_input)
test_x_mean, test_log_var = cvae.encoder.predict(test_input)
val_x_mean, val_log_var = cvae.encoder.predict(val_input)

In [ ]:
embedding(encoded_dim, category_count, train_x_mean, test_x_mean, val_x_mean, train_y, test_y, val_y, train_log_var, test_log_var, val_log_var, labels, quantity = 1000, avg_latent=True)

In [ ]:
reconstructions(cvae, train_x, train_y, train_x_mean, train_log_var, input_label_train, labels, set = 'train')

In [ ]:
reconstructions(cvae, test_x, test_y, test_x_mean, test_log_var, input_label_test, labels, set = 'test')

In [ ]:
import importlib
import generations
importlib.reload(generations)
from generations import Generations

In [ ]:
generator = Generations(cvae, encoded_dim, category_count, input_shape, labels)
generator()

In [ ]:

activations_encoder = VisualizeActivations(cvae, cvae.encoder, test_x, test_y_one_hot)
activations_decoder = VisualizeActivations(cvae, cvae.decoder, test_x, test_y_one_hot)
activations_encoder()
activations_decoder()

In [ ]:
if 'resnet' in model_name:
    target_layer = "layer4"
else:
    target_layer = "block3_conv2"


In [ ]:
gc = GradCam(cvae, test_x, test_y_one_hot, HQ = True, target_layer = target_layer)
gc.gradcam()


In [ ]:
gc.guided_gradcam()

In [ ]:
wandb.finish(exit_code=0, quiet = True) 